In [1]:
!pip install ensemble-boxes
!git clone https://github.com/ultralytics/yolov5.git
!pip install -q yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 7301, done.
remote: Counting objects: 100% (423/423), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 7301 (delta 265), reused 269 (delta 149), pack-reused 6878
Receiving objects: 100% (7301/7301), 9.19 MiB | 7.42 MiB/s, done.
Resolving deltas: 100% (4990/4990), done.


In [2]:
import numpy as np 
import pandas as pd 
import os, cv2, torch
import fastprogress
import shutil as sh

In [3]:
from ensemble_boxes import *

def run_wbf(boxes, scores, image_size=1023, iou_thr=0.5, skip_box_thr=0.7, weights=None):
    labels = [np.zeros(score.shape[0]) for score in scores]
    boxes = [box/(image_size) for box in boxes]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size)
    return boxes, scores, labels

def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

def detect1Image(im0, imgsz, model, device, conf_thres, iou_thres):
    img = letterbox(im0, new_shape=imgsz)[0]
    img = img.transpose(2, 0, 1)  
    img = np.ascontiguousarray(img)

    img = torch.from_numpy(img).to(device)
    img =  img.float()  
    img /= 255.0   
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    pred = model(img, augment=False)[0]

    pred = non_max_suppression(pred, conf_thres, iou_thres)

    boxes = []
    scores = []
    for i, det in enumerate(pred):
        if det is not None and len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
            for *xyxy, conf, cls in det:
                boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
                scores.append(conf)

    return np.array(boxes), np.array(scores) 

In [4]:
paths = [f'../input/wheat-yolo-data-2/best ({i}).pt' for i in range(1, 5)] + ['../input/wheat-yolo-data-2/best.pt']

In [5]:
import yolov5
from yolov5.utils.datasets import *
from yolov5.utils.general import *

In [6]:
source = '../input/aicrowdwheatdata/test/'
imgsz = 1024
conf_thres = 0.5
iou_thres = 0.6
    
fs =  os.listdir(source)
    
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

models = []
for p in paths:
    model = torch.hub.load('yolov5', 'custom', path=p, source='local')
    model.to(device).eval()
    models.append(model)
    
print('models loaded')
    
for f in fastprogress.progress_bar(fs):
    image_id = f[:-4]
    img = cv2.cvtColor(cv2.imread(os.path.join(source, image_id +'.png')), cv2.COLOR_BGR2RGB)
    im_w, im_h = img.shape[:2]
    enboxes = []
    enscores = []
    for model in models:
        for i in range(4):
            im0 = TTAImage(img, i)
            boxes, scores = detect1Image(im0, imgsz, model, device, conf_thres, iou_thres)
            for _ in range(3-i):
                boxes = rotBoxes90(boxes, im_w, im_h)
                    
            enboxes.append(boxes)
            enscores.append(scores) 

        boxes, scores, labels = run_wbf(enboxes, enscores, image_size = im_w, iou_thr=0.6, skip_box_thr=0.43)
        boxes = boxes.astype(np.int32).clip(min=0, max=im_w)
        
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
    boxes = boxes[scores >= 0.05].astype(np.int32)
    scores = scores[scores >=float(0.05)]
        
    lineo = ''
    for box in boxes:
        x1, y1, w, h = box
        xc, yc, w, h = (x1+w/2)/1024, (y1+h/2)/1024, w/1024, h/1024
        lineo += '0 %f %f %f %f\n'%(xc, yc, w, h)
            
    fileo = open(image_id+".txt", 'w+')
    fileo.write(lineo)
    fileo.close()

models loaded
